In [128]:
import pandas as pd
import matplotlib.pyplot as plt

## Chargement des données

In [129]:
fafr = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Query_poly_20210125174741.xlsx", converters={'Matricule':str}) # effectif présent
infoc = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\individus_liste_info_complementaires_20210125175440.xlsx", converters={'Code de la ressource':str}) # données extenes
annuaire = pd.read_csv(r"C:\Users\sgasmi\Desktop\Extractions\Annuaire_20210122.csv", sep=";", encoding ="ANSI", low_memory=False, error_bad_lines=False) # Extraction de l'annuaire
indentité = pd.read_excel(r"C:\Users\sgasmi\Desktop\Interlocuteurs\Gestionnaires d'identité.xlsx")
excl_sal = pd.read_excel(r"C:\Users\sgasmi\Desktop\Interlocuteurs\Salariés à exclure.xlsx")

In [130]:
print(annuaire.shape)
print(annuaire["employeeNumber"])

(10693, 107)
0          80857
1         sansSI
2        A525446
3          31540
4          30332
          ...   
10688      79564
10689      79605
10690      79602
10691      79612
10692      79600
Name: employeeNumber, Length: 10693, dtype: object


In [131]:
## Annuaire, transformation de clé "employeeNumber" en int

def to_int(x):
    try:
        x = int(x)
    except:
        x = 0    
    return x
    

annuaire["employeeNumber"] = annuaire["employeeNumber"].apply(lambda x: to_int(x))
annuaire = annuaire.loc[annuaire["employeeNumber"]!=0]

print(annuaire.shape)

(10560, 107)


In [132]:
excl_sal["Matricule"] = excl_sal["Matricule"].astype("int64")

## Contrôle des données

### Contrôle login SAP

In [133]:
fafr = fafr.merge(infoc[["Code de la ressource", "Login SAP"]], left_on=["Matricule"], right_on=["Code de la ressource"], how='left')

In [134]:
fafr["Login SAP vide"] = ""
fafr.loc[fafr["Login SAP"].isna()==True, "Login SAP vide"] = "Oui"
fafr_sap_ko = fafr.loc[fafr["Login SAP vide"]=="Oui"]
fafr_sap_ko = fafr_sap_ko.merge(indentité, on="Etablissement", how="left")
fafr_sap_ko = fafr_sap_ko.sort_values(by=["Etablissement"])
fafr_sap_ko["Matricule"] =fafr_sap_ko["Matricule"].astype("int64")

In [135]:
fafr_sap_ko = fafr_sap_ko.merge(annuaire[["employeeNumber", "saplogon"]],left_on="Matricule", right_on="employeeNumber", how="left")

In [136]:
fafr_sap_ko = fafr_sap_ko[["Matricule","Nom", "Prénom", "Etablissement","Gestionnaire d'identité", "employeeNumber", "saplogon"]]
fafr_sap_ko = fafr_sap_ko.loc[fafr_sap_ko["Matricule"].isin(excl_sal["Matricule"])==False]
sap_ko_annuaire = fafr_sap_ko.loc[fafr_sap_ko["saplogon"].isna()]

## Export

### Export login SAP KO

In [137]:
with pd.ExcelWriter(r'C:\Users\sgasmi\Desktop\Export Python\SAP_KO.xlsx') as writer:  
    fafr_sap_ko.to_excel(writer, sheet_name='Conso globale', index=False)
    sap_ko_annuaire.to_excel(writer, sheet_name='SAP Manquant dans annuaire', index=False)


